In [5]:
import os
import pickle
import boto3
import gzip
from io import BytesIO
#import joblib
import sagemaker
import sagemaker.session

from sagemaker.transformer import Transformer
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput,TransformInput,CreateModelInput
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.conditions import (
    ConditionGreaterThanOrEqualTo,
)
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)
from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
)
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import (
    ProcessingStep,
    TrainingStep,
    TransformStep,
    CreateModelStep
)
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.model import Model
from sagemaker.network import NetworkConfig

from datetime import datetime
from dateutil import tz
import json
import boto3
from sagemaker.workflow.parameters import (
        ParameterInteger,
        ParameterString,
    )

sess = sagemaker.Session()

import pytz
from datetime import datetime

def convert_timezone():
    to_zone = tz.gettz('Asia/Kolkata')
    
    return datetime.utcnow().astimezone(to_zone)

   
def get_session(region, default_bucket):
    """Gets the sagemaker session based on the region.
    Args:
        region: the aws region to start the session
        default_bucket: the bucket to use for storing the artifacts
    Returns:
        `sagemaker.session.Session instance
    """

    boto_session = boto3.Session(region_name=region)

    sagemaker_client = boto_session.client("sagemaker")
    runtime_client = boto_session.client("sagemaker-runtime")
    return sagemaker.session.Session(
        boto_session=boto_session,
        sagemaker_client=sagemaker_client,
        sagemaker_runtime_client=runtime_client,
        default_bucket=default_bucket,
    )



## Preprocessing, Training & Register Model

In [6]:
def get_pipeline(
    region,
    subnets,
    security_group_ids,
    role=None,
    default_bucket=None,
    model_package_group_name='mlops',
    pipeline_name='mlops-prototype-training-pipeline',  # You can find your pipeline name in the Studio UI (project -> Pipelines -> name)
    base_job_prefix='lseg-mlops',
    env='dev'
):
    
    
    srilanka_tz = pytz.timezone('Asia/Colombo')
    s3 = boto3.client('s3')
    #working input data path
    input_data = 's3://sagemaker-studio-408970228172-8k2laox6686/data-ingestion/'
    #input_data2 = config['tr_preprocessing']['inputs']['input2']
    #input_data3 = config['tr_preprocessing']['inputs']['input3']
    #input_data4 = config['tr_preprocessing']['inputs']['input4']
    
    date_folder = datetime.now(srilanka_tz).strftime("%Y-%m-%d")
       
    sagemaker_session = get_session(region, default_bucket)
    if role is None:
        role = sagemaker.session.get_execution_role(sagemaker_session)
    account_id = boto3.client("sts").get_caller_identity().get("Account")
    region = boto3.session.Session().region_name

    # Parameters for pipeline execution
    
    preprocess_output_data1 = "s3://sagemaker-studio-408970228172-8k2laox6686/preprocess/train/"
    #preprocess_output_data2 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}{project_name}/{job_name}/{date_folder}/preprocess_output_data/output2"
    #preprocess_output_data3 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/{job_name}/{date_folder}/preprocess_output_data/output3"
    #preprocess_output_data4 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/{job_name}/{date_folder}/preprocess_output_data/output4"
    
#     model_path = ParameterString(
#         name="ModelPath",
#         default_value= f"s3://sagemaker-studio-408970228172-8k2laox6686/train/models/", 
#     )
        
    model_approval_status = ParameterString(
        name="ModelApprovalStatus",
        default_value="Approved",  # ModelApprovalStatus can be set to a default of "Approved" if you don't want manual approval.
    )     
 
    # generic_tags=[{'Key': 'EnvironmentName', 'Value': env}, 
    #               {'Key': 'ProjectName', 'Value': config['pipeline']['generic_tags']['ProjectName']},
    #               {'Key': 'DepartmentName', 'Value': config['pipeline']['generic_tags']['DepartmentName']}, 
    #               {'Key': 'UsecaseName', 'Value': config['pipeline']['generic_tags']['UsecaseName']},
    #               {'Key': 'ResourceName', 'Value': config['pipeline']['generic_tags']['ResourceName']}, 
    #               {'Key': 'OwnerName', 'Value': config['pipeline']['generic_tags']['OwnerName']},
    #               {'Key': 'BUName', 'Value': config['pipeline']['generic_tags']['BUName']} ]
    

    # --------------------- PREPROCESSING --------------------------------------------------------------------
    ecr_repository = ''
    tag = ":latest"
    uri_suffix = "amazonaws.com"
    
    preprocessing_repository_uri = "{}.dkr.ecr.{}.{}/{}".format(
        account_id, region, uri_suffix, ecr_repository + tag
    )
        
    script_processor = ScriptProcessor(
         command = ["python3"],
         image_uri = '408970228172.dkr.ecr.ap-southeast-1.amazonaws.com/mlops-prototype-preprocessing-image@sha256:33087f4aed9b09ecbb6997a791b0ea34fc1667b7fd10e7345f4d557289d9bdad',
         role = role,
         instance_count = 1,
         instance_type = "ml.t3.medium",
         #tags = generic_tags + [{'Key': 'JobType', 'Value': 'Preprocessing'}],
         tags = [{'Key': 'JobType', 'Value': 'Preprocessing'}],
         #network_config = NetworkConfig(subnets=subnets.split(':'), security_group_ids=security_group_ids.split(':'))
    )
    
    step_preprocess = ProcessingStep(
        name=f"{base_job_prefix}-preprocessing",
        processor=script_processor, 
        code='mlops_prototype_tr_preprocessing/preprocessing.py',
        inputs=[ProcessingInput(input_name="input-1",source=input_data, destination="/opt/ml/processing/input"),
               #ProcessingInput(input_name="input-2", source=input_data2, destination="/opt/ml/processing/input2"),
                #ProcessingInput(input_name="input-3", source=input_data3, destination="/opt/ml/processing/input3"),
                #ProcessingInput(input_name="input-4", source=input_data4, destination="/opt/ml/processing/input4"),
               ],
        outputs=[
            ProcessingOutput(output_name="output1", destination=preprocess_output_data1, source="/opt/ml/processing/output1"),
            #ProcessingOutput(output_name="output2", destination=preprocess_output_data2, source="/opt/ml/processing/output2"),
            #ProcessingOutput(output_name="output3", destination=preprocess_output_data3, source="/opt/ml/processing/output3"),
            #ProcessingOutput(output_name="output4", destination=preprocess_output_data4, source="/opt/ml/processing/output4"),
        ]
    )
    
    
            
#     script_processor2 = ScriptProcessor(
#          command = ["python3"],
#          image_uri = '408970228172.dkr.ecr.ap-southeast-1.amazonaws.com/mlops-prototype-preprocessing-image@sha256:33087f4aed9b09ecbb6997a791b0ea34fc1667b7fd10e7345f4d557289d9bdad',
#          role = role,
#          instance_count = 1,
#          instance_type = "ml.t3.medium",
#          #tags = generic_tags + [{'Key': 'JobType', 'Value': 'Preprocessing'}],
#          #tags = [{'Key': 'JobType', 'Value': 'train'}],
#          #network_config = NetworkConfig(subnets=subnets.split(':'), security_group_ids=security_group_ids.split(':'))
#     )
    
#     step_train = ProcessingStep(
#         name=f"{base_job_prefix}-training",
#         processor=script_processor2, 
#         #code='mlops_prototype_tr_training/model/train.py',
#         code='mlops_prototype_tr_preprocessing/preprocessing.py',
#         #inputs=[ProcessingInput(input_name="input-1",source=input_data, destination="/opt/ml/processing/input"),],
#         inputs=[ProcessingInput(input_name="input-1", source=step_preprocess.properties.ProcessingOutputConfig.Outputs["output1"].S3Output.S3Uri, destination="/opt/ml/processing/input4")],
#         #tags=[{'Key': 'JobType', 'Value': 'train'}]
#     )

    
    
    ##--------------------------------------- Training Job-----------------------------------------------------

    ecr_repository = ''
    tag = ":latest"
    uri_suffix = "amazonaws.com"
    
    recommender_image_uri = "{}.dkr.ecr.{}.{}/{}".format(
        account_id, region, uri_suffix, ecr_repository + tag
    )
    
    estimator = Estimator(image_uri='408970228172.dkr.ecr.ap-southeast-1.amazonaws.com/mlops-prototype-training-image3:latest',
                      role=role,
                      sagemaker_session=sess,
                      output_path='s3://sagemaker-studio-408970228172-8k2laox6686/train',
                      base_job_name=f"{base_job_prefix}-training-job",
                      input_mode='File',                                     
                      instance_count=2,
                      instance_type='ml.m5.large',
                      debugger_hook_config=False,
                      disable_profiler = True,
                      metric_definitions=[
                          # You can define your own regex
                            {'Name': 'Accuracy: ' , 'Regex': 'Accuracy: ([-+]?[0-9]*\.?[0-9]+)'},
                            {'Name': 'Precision: ' , 'Regex': 'Precision: ([-+]?[0-9]*\.?[0-9]+)'},
                            {'Name': 'Recall: ' , 'Regex': 'Recall: ([-+]?[0-9]*\.?[0-9]+)'},
                            {'Name': 'F1 score: ' , 'Regex': 'f1: ([-+]?[0-9]*\.?[0-9]+)'},
                            {'Name': 'Latency: ' , 'Regex': 'Latency: ([-+]?[0-9]*\.?[0-9]+)'},
                      ],
                      #subnets = subnets.split(':'),
                      #security_group_ids = security_group_ids.split(':')
                         )

    # start training
    step_train = TrainingStep(
        name=f"{base_job_prefix}-training",
        estimator=estimator,
        inputs = {"training": TrainingInput(
                s3_data='s3://sagemaker-studio-408970228172-8k2laox6686/preprocess/train/', #step_preprocess.properties.ProcessingOutputConfig.Outputs["output1"].S3Output.S3Uri,
                content_type="text/csv",
            ),
                 # "training2": TrainingInput(
                #s3_data=step_preprocess.properties.ProcessingOutputConfig.Outputs["output2"].S3Output.S3Uri,
                #content_type="text/csv",
            #),
                   # "training3": TrainingInput(
                #s3_data=step_preprocess.properties.ProcessingOutputConfig.Outputs["output3"].S3Output.S3Uri,
                #content_type="text/csv",
            #),
                   # "training4": TrainingInput(
                #s3_data=step_preprocess.properties.ProcessingOutputConfig.Outputs["output4"].S3Output.S3Uri,
                #content_type="text/csv",
            #),
        }
    ) 

    ###### --------------------- Model Registry ----------------------------------------------------------------
    """
    #registering the model

    step_register = RegisterModel(
        name= f"{base_job_prefix}-registerxgbmodel",
        estimator= estimator,
        model_data= step_train.properties.ModelArtifacts.S3ModelArtifacts,
        content_types= ["text/csv"],
        response_types= ["text/csv"],
        inference_instances= ["ml.t2.medium", "ml.m5.xlarge"],
        transform_instances= ["ml.m5.xlarge"],
        model_package_group_name=f"{model_package_group_name}-XGB",
        approval_status=model_approval_status,
        #model_metrics=model_metrics,
    )
   """
    # Pipeline instance
    pipeline = Pipeline(
        name=pipeline_name+env,
        parameters=[
            env,
            #model_approval_status,
            #model_path
        ],
        steps=[#step_preprocess,
               step_train,
               #step_register
              ],
        sagemaker_session=sagemaker_session,
    )
    return pipeline

In [7]:
#new subnet
dev_subnets = 'subnet-0291c313148952e85'
dev_sg = 'sg-0ae3a97e3f729b7e6'


model_package_group_name='mlops-prototype'
pipeline_name='mlops-prototype-tr-pripeline' # You can find your pipeline name in the Studio UI (project -> Pipelines -> name)
base_job_prefix='lseg-mlops'
env='dev'

region ='ap-southeast-1'
role='arn:aws:iam::408970228172:role/service-role/AmazonSageMaker-ExecutionRole-20230323T125466'
default_bucket='sagemaker-studio-408970228172-8k2laox6686'
pipeline_def = get_pipeline(
    region=region,
    subnets = dev_subnets,
    security_group_ids = dev_sg,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
    env='dev',
)



In [8]:
pipeline_def.upsert(role_arn=role)

execution = pipeline_def.start()

In [9]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-southeast-1:408970228172:pipeline/mlops-prototype-tr-pripelinedev',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-southeast-1:408970228172:pipeline/mlops-prototype-tr-pripelinedev/execution/v1zi27l3x2ai',
 'PipelineExecutionDisplayName': 'execution-1680269773878',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'mlops-prototype-tr-pripelinedev',
  'TrialName': 'v1zi27l3x2ai'},
 'CreationTime': datetime.datetime(2023, 3, 31, 13, 36, 13, 771000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 3, 31, 13, 36, 13, 771000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '4a0a7271-9024-4fad-bd8c-69b4604d9b94',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4a0a7271-9024-4fad-bd8c-69b4604d9b94',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '544',
   'date': 'Fri, 31 Mar 2023 13:36:15 GMT'},
  'RetryAttempts': 0}

------------------------

In [35]:
import pandas as pd
train_features_data = 's3://sagemaker-studio-408970228172-8k2laox6686/preprocess/train/x.csv'
X = pd.read_csv(train_features_data, header = None)

In [36]:
X

0      1     2     3      4     5      6     7
0     6.0  148.0  72.0  35.0    0.0  33.6  0.627  50.0
1     1.0   85.0  66.0  29.0    0.0  26.6  0.351  31.0
2     8.0  183.0  64.0   0.0    0.0  23.3  0.672  32.0
3     1.0   89.0  66.0  23.0   94.0  28.1  0.167  21.0
4     0.0  137.0  40.0  35.0  168.0  43.1  2.288  33.0
..    ...    ...   ...   ...    ...   ...    ...   ...
763  10.0  101.0  76.0  48.0  180.0  32.9  0.171  63.0
764   2.0  122.0  70.0  27.0    0.0  36.8  0.340  27.0
765   5.0  121.0  72.0  23.0  112.0  26.2  0.245  30.0
766   1.0  126.0  60.0   0.0    0.0  30.1  0.349  47.0
767   1.0   93.0  70.0  31.0    0.0  30.4  0.315  23.0

[768 rows x 8 columns]

In [41]:
    train_features_data = 's3://sagemaker-studio-408970228172-8k2laox6686/preprocess/train/x.csv'
    train_labels_data = 's3://sagemaker-studio-408970228172-8k2laox6686/preprocess/test/y.csv'
    
    
    print("Reading input data")
    print("Reading input data from {}".format(train_features_data))
    X = pd.read_csv(train_features_data, header = None)
    print("Reading input data successfull")

Reading input data
Reading input data from s3://sagemaker-studio-408970228172-8k2laox6686/preprocess/train/x.csv
Reading input data successfull


In [42]:
X

0      1     2     3      4     5      6     7
0     6.0  148.0  72.0  35.0    0.0  33.6  0.627  50.0
1     1.0   85.0  66.0  29.0    0.0  26.6  0.351  31.0
2     8.0  183.0  64.0   0.0    0.0  23.3  0.672  32.0
3     1.0   89.0  66.0  23.0   94.0  28.1  0.167  21.0
4     0.0  137.0  40.0  35.0  168.0  43.1  2.288  33.0
..    ...    ...   ...   ...    ...   ...    ...   ...
763  10.0  101.0  76.0  48.0  180.0  32.9  0.171  63.0
764   2.0  122.0  70.0  27.0    0.0  36.8  0.340  27.0
765   5.0  121.0  72.0  23.0  112.0  26.2  0.245  30.0
766   1.0  126.0  60.0   0.0    0.0  30.1  0.349  47.0
767   1.0   93.0  70.0  31.0    0.0  30.4  0.315  23.0

[768 rows x 8 columns]